# Pràctica 2

In [1]:
import nltk
from nltk.corpus import movie_reviews as mr
from nltk.corpus import wordnet as wn

Downloads

In [13]:
#nltk.download('movie_reviews')
#nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

Visualització dels textos

In [ ]:
file_ids = mr.fileids('pos')

# Imprimir el texto de los archivos
for file_id in file_ids:
    print("Texto del archivo:", file_id)
    print(mr.raw(file_id))
    print("\n-----------------------\n")

In [9]:
mr.words('pos/cv000_29590.txt')

['films', 'adapted', 'from', 'comic', 'books', 'have', ...]

## Preprocessing

In [89]:
len(file_ids)

1000

In [90]:
# train i test
train_pos = []
test_pos = []

i = 0
while i < 700:
    train_pos.append(nltk.pos_tag(mr.words(file_ids[i])))
    i += 1
    
while i < 999:
    test_pos.append(nltk.pos_tag(mr.words(file_ids[i])))
    i += 1

### Eliminem els signes de puntuació

In [98]:
import re

def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

new_train = []

for phrase in train_pos:
    afegim = []
    for word in phrase:
        if not word[0].isdigit() and remove_punctuation(word[0]) == word[0]: # Verificar si la palabra no contiene ningún signo de puntuación
            afegim.append(word)
        #else:
            #print('eliminem: ', word)
    new_train.append(afegim)
# Sobrescribir la lista original con la nueva lista de palabras procesadas
#tagger = new_tagger

In [99]:
new_test = []

for phrase in test_pos:
    afegim = []
    for word in phrase:
        if not word[0].isdigit() and remove_punctuation(word[0]) == word[0]: # Verificar si la palabra no contiene ningún signo de puntuación
            afegim.append(word)
    
    new_test.append(afegim)
# Sobrescribir la lista original con la nueva lista de palabras procesadas
#test = new_tagger

### Lematització

In [69]:
wnl = nltk.stem.WordNetLemmatizer()

def lemmatize(p):
  d = {'NN': 'n', 'NNS': 'n', 'NNP': 'n', 'NNPS': 'n',
       'JJ': 'a', 'JJR': 'a', 'JJS': 'a', 
       'VB': 'v', 'VBD': 'v', 'VBG': 'v', 'VBN': 'v', 'VBP': 'v', 'VBZ': 'v', 
       'RB': 'r', 'RBR': 'r', 'RBS': 'r'}
  if p[1] in d:
    return wnl.lemmatize(p[0], pos=d[p[1]])
  return p[0]

In [ ]:
new_tagger = []

for i, word in enumerate(tagger):
    print('antes: ', word)
    new_tagger.append((lemmatize(word), word[1]))
    print('despues:', new_tagger[i])

## Hidden Markov Model

In [103]:
trainer = nltk.tag.hmm.HiddenMarkovModelTrainer()
HMM = trainer.train_supervised(new_train)

HMM.accuracy(new_test)

c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\nltk\tag\hmm.py:334: RuntimeWarning: overflow encountered in cast
  X[i, j] = self._transitions[si].logprob(self._states[j])
c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\nltk\tag\hmm.py:336: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])
c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\nltk\tag\hmm.py:332: RuntimeWarning: overflow encountered in cast
  P[i] = self._priors.logprob(si)
c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\nltk\tag\hmm.py:364: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])


0.11829235367395949

Afegim funció de smoothing

In [104]:
def LID(fd, bins):
  return nltk.probability.LidstoneProbDist(fd, 0.1, bins)
trainer = nltk.tag.hmm.HiddenMarkovModelTrainer()

HMM = trainer.train_supervised(new_train, estimator=LID)
HMM.accuracy(new_test)

0.8944302082691916

In [105]:
HMM = nltk.HiddenMarkovModelTagger.train(new_train)
HMM.accuracy(new_test)

0.8944302082691916

## Pràctica 2.a

In [3]:
print('Dos primers fitxers: ', mr.fileids('pos')[:2])
print('Número de textos amb connotació negativa: ', len(mr.fileids('neg')))
print('Head de paraules en el primer text amb connotació positiva: ', mr.words(mr.fileids('pos')[0]))

Dos primers fitxers:  ['pos/cv000_29590.txt', 'pos/cv001_18431.txt']
Número de textos amb connotació negativa:  1000
Head de paraules en el primer text amb connotació positiva:  ['films', 'adapted', 'from', 'comic', 'books', 'have', ...]


In [56]:
text = mr.raw(mr.fileids('pos')[0])

frases = re.split(r'[\n]', text)

# Imprimir les frases
for frase in frases:
    print('yes:', frase.strip())  # Utilitzem strip() per eliminar els espais en blanc addicionals al principi i al final de cada frase

yes: films adapted from comic books have had plenty of success , whether they're about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there's never really been a comic book like from hell before .
yes: for starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid '80s with a 12-part series called the watchmen .
yes: to say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd .
yes: the book ( or " graphic novel , " if you will ) is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes .
yes: in other words , don't dismiss this film because of its source .
yes: if you can get past the whole comic book thing , you might find another stumbling block in from hell's directors , albert and allen hughes .
yes: getting the hughes brothers to

### Preprocessing

In [ ]:
import re

def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)


# Afegim només les paraules de cada text en una nova llista pels textos amb connotació positiva
positius = []
for i in range(len(mr.fileids('pos'))):
    afegim = []
    for word in mr.words(mr.fileids('pos')[i]):
        if not word.isdigit() and remove_punctuation(word) == word: # Verificar si la palabra no contiene ningún signo de puntuación o dígito
            afegim.append(word)
            
    positius.append(afegim)
    
    
# Afegim només les paraules de cada text en una nova llista pels textos amb connotació negativa
negatius = []
for i in range(len(mr.fileids('neg'))):
    afegim = []
    for word in mr.words(mr.fileids('neg')[i]):
        if not word.isdigit() and remove_punctuation(word) == word: # Verificar si la palabra no contiene ningún signo de puntuación o dígito
            afegim.append(word)
            
    negatius.append(afegim)

El bo:

In [2]:
# Obtenir les opinions del corpus de Movie Reviews
documents = [(mr.raw(fileid), category)
             for category in mr.categories()
             for fileid in mr.fileids(category)]

etiquetes = [opinion[1] for opinion in documents]

In [3]:
import re

def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

textos = []
afegim = []
for opinio in documents:
    # Separa les paraules del text sense tenir en compte els salts de línia
    words = re.split(r'\s+', opinio[0])
    for word in words:
        if not word.isdigit() and remove_punctuation(word) == word: # Verificar si la palabra no contiene ningún signo de puntuación o dígito
            afegim.append(word)

    textos.append(' '.join(afegim))

In [4]:
from sklearn.model_selection import train_test_split, cross_val_score

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(textos, etiquetes, test_size=0.2, random_state=42)

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

# Vectorització dels textos
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_train)

In [ ]:
count = {}
for i in X.toarray():
    for index, j in enumerate(i):
        count[index] = count.get(index, 0) + j
        
print(count)

In [ ]:
import numpy as np

index_eliminar = {index: freq for index, freq in count.items() if freq < 10}

matriu = np.delete(X, index_eliminar, axis=1)



In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score


np.random.seed(42)

# Genera 100 índices aleatorios únicos dentro del rango de 0 a 2000
indices_train = np.random.choice(range(2000), size=150, replace=False)

train_textos = [textos[i] for i in indices_train]
train_etiquetes = [etiquetes[i] for i in indices_train]

"""
np.random.seed(32)

indices_test = np.random.choice(range(2000), size=20, replace=False)

test_textos = [textos[i] for i in indices_test]
test_etiquetes = [etiquetes[i] for i in indices_test]"""

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(train_textos, train_etiquetes, test_size=0.2, random_state=42)


# Vectorització dels textos
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_train)

#print("---"*20)
#print("Visualitzacio de vectorizer: ", vectorizer.get_feature_names_out())
#print('\n')
#print("---"*20)

# Entrenament del model Naive Bayes amb validació creuada (cross-validation)
naive_bayes_model = MultinomialNB()
nb_scores = cross_val_score(naive_bayes_model, X, y_train, cv=5)
naive_bayes_model = naive_bayes_model.fit(X, y_train)

# Entrenament del model SVM amb validació creuada (cross-validation)
svm_model = SVC(kernel='linear')
svm_scores = cross_val_score(svm_model, X, y_train, cv=5)
svm_model = svm_model.fit(X, y_train)

# Resultats
print("Precisió del model Naive Bayes amb validació creuada:", nb_scores.mean())
print("Precisió del model SVM amb validació creuada:", svm_scores.mean())

print("---"*20)


nb_predict = naive_bayes_model.predict(X_test)
precisio_nb = accuracy_score(y_test, nb_predict)
print("Precisió del model Naive Bayes:", precisio_nb)

svm_predict = svm_model.predict(X_test)
precisio_svm = accuracy_score(y_test, svm_predict)
print("Precisió del model SVM:", precisio_svm)

In [12]:
import warnings

# Desactivar todos los warnings
warnings.filterwarnings("ignore")


In [22]:
import nltk
import random
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix


# Obtener las revisiones y sus categorías (pos/neg)
documents = [(list(mr.words(fileid)), category)
             for category in mr.categories()
             for fileid in mr.fileids(category)]

# Mezclar los documentos para garantizar una distribución aleatoria
random.shuffle(documents)

# Preprocesamiento: unir las palabras y eliminar stopwords y signos de puntuación
#stopwords = set(nltk.corpus.stopwords.words('english'))
punctuation = set(string.punctuation)
preprocessed_documents = []
for words, category in documents:
    words = [word.lower() for word in words if word.lower() not in punctuation]
    preprocessed_documents.append((' '.join(words), category))

# Separar los datos en características (X) y etiquetas (y)
X = [text for text, category in preprocessed_documents]
y = [category for text, category in preprocessed_documents]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un pipeline con CountVectorizer y un clasificador
pipeline_NB = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', MultinomialNB())
])

pipeline_LR = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression())
])

# Validación cruzada para obtener la precisión media del modelo NB
scores_NB = cross_val_score(pipeline_NB, X_train, y_train, cv=5)
print("Precisión (accuracy) media con Naive Bayes:", scores_NB.mean())

# Entrenar y evaluar el modelo con Naive Bayes
pipeline_NB.fit(X_train, y_train)
y_pred_NB = pipeline_NB.predict(X_test)
print("Precisión (accuracy) con Naive Bayes:", accuracy_score(y_test, y_pred_NB))
print("Matriz de confusión con Naive Bayes:")
print(confusion_matrix(y_test, y_pred_NB))

# Validación cruzada para obtener la precisión media del modelo LR
scores_LR = cross_val_score(pipeline_LR, X_train, y_train, cv=5)
print("Precisión (accuracy) media con Logistic Regression:", scores_LR.mean())

# Entrenar y evaluar el modelo con Logistic Regression
pipeline_LR.fit(X_train, y_train)
y_pred_LR = pipeline_LR.predict(X_test)
print("Precisión (accuracy) con Logistic Regression:", accuracy_score(y_test, y_pred_LR))
print("Matriz de confusión con Logistic Regression:")
print(confusion_matrix(y_test, y_pred_LR))


Precisión (accuracy) media con Naive Bayes: 0.8125
Precisión (accuracy) con Naive Bayes: 0.82
Matriz de confusión con Naive Bayes:
[[162  36]
 [ 36 166]]
Precisión (accuracy) media con Logistic Regression: 0.835
Precisión (accuracy) con Logistic Regression: 0.835
Matriz de confusión con Logistic Regression:
[[160  38]
 [ 28 174]]


In [27]:
len(X_test)

400

### Count Vectorizer

Ho tornem a fer tot pero ara amb count vectorizer

In [106]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
for file_id in file_ids:
    print(file_id)

In [114]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(mr.words(file_id) for file_id in file_ids)

AttributeError: 'StreamBackedCorpusView' object has no attribute 'lower'

In [111]:
vectorizer.get_feature_names_out()

array(['cv000_29590', 'cv001_18431', 'cv002_15918', ..., 'cv999_13106',
       'pos', 'txt'], dtype=object)